In [1]:
from google.colab import files

# Upload file kaggle.json
print("Silakan upload file 'kaggle.json' yang sudah Anda download.")
files.upload()

Silakan upload file 'kaggle.json' yang sudah Anda download.


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rizanursyah","key":"8a8af6a9dc28cdebe0a0ff42d5244abf"}'}

In [2]:
# Buat direktori .kaggle dan siapkan API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset Mental Health FAQ dari Kaggle
!kaggle datasets download -d narendrageek/mental-health-faq-for-chatbot

# Unzip dataset yang baru di-download
# Nama file zip-nya akan sesuai dengan nama dataset
!unzip mental-health-faq-for-chatbot.zip

Dataset URL: https://www.kaggle.com/datasets/narendrageek/mental-health-faq-for-chatbot
License(s): other
  0% 0.00/50.1k [00:00<?, ?B/s]
100% 50.1k/50.1k [00:00<00:00, 183MB/s]
Archive:  mental-health-faq-for-chatbot.zip
  inflating: Mental_Health_FAQ.csv   


In [1]:
import pandas as pd

# Path ke file CSV yang sudah di-unzip
file_path = 'Mental_Health_FAQ.csv'

# Load data menjadi DataFrame
df = pd.read_csv(file_path)

In [2]:
# Hapus kolom yang tidak kita perlukan
df = df.drop('Question_ID', axis=1)

In [3]:
# Ganti nama kolom
df.rename(columns={'Questions': 'pertanyaan', 'Answers': 'jawaban'}, inplace=True)

# Tampilkan 5 baris pertama untuk verifikasi
print("\nDataset FAQ Kesehatan Mental berhasil di-load!")
df.head()


Dataset FAQ Kesehatan Mental berhasil di-load!


,pertanyaan,jawaban
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [4]:
!pip install -q sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer

# Gunakan model multilingual yang populer dan efisien
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'

# Load model dari Hugging Face.
# Proses ini akan men-download modelnya, mungkin butuh beberapa saat.
print("Mulai men-download model embedding... Harap tunggu.")
embedding_model = SentenceTransformer(model_name)
print("Model embedding berhasil di-load!")

Mulai men-download model embedding... Harap tunggu.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model embedding berhasil di-load!


In [6]:
# Kode yang sudah diperbaiki
list_of_questions = df['pertanyaan'].tolist()

# Tambahkan convert_to_tensor=True di sini!
question_embeddings = embedding_model.encode(list_of_questions, convert_to_tensor=True)

print("Database pertanyaan berhasil di-encode ulang dan dikirim ke GPU.")

Database pertanyaan berhasil di-encode ulang dan dikirim ke GPU.


In [7]:
# --- Cek Hasilnya ---
print("\nProses encoding selesai!")
print("Bentuk (shape) dari array embeddings kita:", question_embeddings.shape)

# Kita juga bisa lihat contoh satu embedding
print("\nContoh 'DNA Makna' untuk pertanyaan pertama:")
print(question_embeddings[0])


Proses encoding selesai!
Bentuk (shape) dari array embeddings kita: torch.Size([98, 384])

Contoh 'DNA Makna' untuk pertanyaan pertama:
tensor([ 4.4400e-01, -4.4506e-02,  2.3824e-01,  6.6620e-01,  1.2571e-02,
        -7.6353e-02,  5.0787e-01, -8.1078e-03,  2.4119e-01, -6.6416e-02,
         9.7993e-02, -1.4159e-01,  7.7150e-02, -4.6925e-02,  2.4467e-01,
         1.4484e-01,  9.1241e-02, -3.8281e-01, -3.5446e-02,  1.7019e-01,
         1.6075e-01, -4.3232e-03,  1.3737e-01, -3.3878e-01,  1.3477e-01,
         2.9897e-01, -5.0048e-02, -2.5144e-01, -2.3268e-01,  3.8959e-01,
        -1.9392e-01,  3.3499e-01, -1.6688e-01,  7.9944e-02,  9.2190e-02,
         1.3870e-01, -2.3108e-01,  3.4266e-01,  1.5883e-01, -2.4133e-01,
         7.0319e-02,  3.5755e-01,  3.5637e-01, -6.3357e-05, -2.7982e-01,
         1.8291e-01, -1.4055e-01,  1.1946e-01,  1.8065e-01, -2.8937e-01,
        -5.2816e-02,  3.6580e-01,  6.8330e-01,  4.6210e-02,  4.5181e-02,
         1.9026e-01,  2.5346e-01, -1.7530e-01, -3.2774e-01, 

In [17]:
from sentence_transformers import util
import torch

# 'embedding_model' dan 'question_embeddings' adalah variabel dari langkah sebelumnya.

def find_best_answer(user_question):
    """Find most similar answer from FAQ database"""
    query_embedding = embedding_model.encode(user_question, convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_embedding, question_embeddings)
    best_match_index = torch.argmax(cosine_scores).item()
    best_answer = df['jawaban'].iloc[best_match_index]
    best_score = torch.max(cosine_scores).item()
    return best_answer, best_score

In [18]:
# --- Mari Kita Lakukan Tes! ---

# Tulis pertanyaan lo di sini. Coba pake kalimat yang beda
# dari yang ada di dataset untuk melihat keajaibannya.
user_question = "Aku merasa gelisah saat sendirian"

# Cari jawaban terbaik
answer = find_best_answer(user_question)

# Tampilkan hasilnya
print("Pertanyaan User:", user_question)
print("\nJawaban yang Ditemukan:")
print(answer)

Pertanyaan User: Aku merasa gelisah saat sendirian

Jawaban yang Ditemukan:
("A lot of people are alone right now, but we don't have to be lonely. We're all in this together. \n While you may be physically separated from friends, family members, and other loved ones, it has never been more important to maintain those social connections. Social connections are an opportunity to seek and share support, talk through difficult feelings, share a laugh, keep up-to-date with loved ones, and help each other cope. This pandemic is a lot for one person to deal with on their own. While measures like physical distancing and self-isolation are necessary to slow the spread of the virus, the physical separation can amplify a lot of challenging emotions like loneliness and fear. \n Think about the different ways to connect that are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one set time for a video c

In [19]:
!pip install -q -U langchain langchain-community

In [20]:
from langchain.prompts import PromptTemplate
from google.colab import userdata
import requests
import torch

In [21]:
# 1. TEMPLATE PROMPT
# Definisikan "perintah" untuk LLM di satu tempat biar rapi.
PROMPT_TEMPLATE = """
Anda adalah asisten AI "Mentor AI" yang ramah dan profesional, menjawab pertanyaan seputar kesehatan mental.
Jawab pertanyaan pengguna HANYA berdasarkan konteks yang diberikan di bawah ini.
Jika informasi tidak ada di konteks, katakan dengan sopan bahwa Anda belum memiliki informasi spesifik tentang itu, tapi berikan saran umum yang relevan jika memungkinkan.
Jawab dengan gaya bahasa yang natural, membantu, dan empatik dalam Bahasa Indonesia.

KONTEKS:
{context}

PERTANYAAN PENGGUNA:
{question}

JAWABAN ANDA:
"""

In [22]:
# 2. FUNGSI PENCARI KONTEKS (Retriever)
# Fungsi find_best_answer yang lama, tapi kita pastikan dia hanya mengembalikan teks jawaban.
def find_best_answer(user_question):
    query_embedding = embedding_model.encode(user_question, convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_embedding, question_embeddings)
    best_match_index = torch.argmax(cosine_scores).item()
    best_answer = df['jawaban'].iloc[best_match_index]
    return best_answer

In [23]:
# 3. FUNGSI GENERATOR JAWABAN (LLM)
def generate_llm_response(user_question, context):
    try:
        api_key = userdata.get('GROQ_API_KEY')
        if not api_key:
            print("⚠️ GROQ_API_KEY tidak ditemukan di Colab Secrets!")
            return None
    except Exception as e:
        print(f"Error saat mengambil Groq API Key: {e}")
        return None

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Buat prompt dari template
    prompt_text = PROMPT_TEMPLATE.format(context=context, question=user_question)

    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt_text}],
        "temperature": 0.7,
        "max_tokens": 300
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        response.raise_for_status() # Ini akan memunculkan error jika status code bukan 2xx
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"Error koneksi ke Groq API: {e}")
        return None
    except Exception as e:
        print(f"Error saat memproses response Groq: {e}")
        return None

In [24]:
# 4. FUNGSI UTAMA YANG MERANGKAI SEMUANYA
def get_chatbot_response(user_question):
    print(f"❓ PERTANYAAN DITERIMA: {user_question}")
    print("--------------------------------------------------")

    # Langkah 1: Ambil konteks yang relevan
    print("🔍 Mencari konteks di database...")
    context = find_best_answer(user_question)
    print("✅ Konteks ditemukan!")

    # Langkah 2: Suruh LLM buat jawaban berdasarkan konteks
    print("🤖 'Mentor AI' sedang berpikir...")
    llm_answer = generate_llm_response(user_question, context)

    # Langkah 3: Berikan jawaban akhir
    if llm_answer:
        final_answer = llm_answer
    else:
        # Fallback jika LLM gagal, kita tetap kasih jawaban dari database
        print("🔄 LLM gagal merespon, menggunakan jawaban langsung dari database.")
        final_answer = context

    return final_answer

In [25]:
# --- TESTING DENGAN STRUKTUR BARU ---

user_question = "Bagaimana cara mengatasi stres kerja?"
final_answer = get_chatbot_response(user_question)

print("\n================ JAWABAN FINAL ==================")
print(final_answer)

❓ PERTANYAAN DITERIMA: Bagaimana cara mengatasi stres kerja?
--------------------------------------------------
🔍 Mencari konteks di database...
✅ Konteks ditemukan!
🤖 'Mentor AI' sedang berpikir...

================ JAWABAN FINAL ==================
Stres kerja! It's a common issue many of us face. As a friendly AI, I'd like to offer some suggestions to help you manage work-related stress.

Firstly, it's essential to acknowledge that stress is a normal response to external pressures, and it's not uncommon to feel overwhelmed at work. Here are some strategies that may help:

1. **Identify the sources of stress**: Take some time to reflect on what specifically is causing your stress at work. Is it a particular task, colleague, or workload? Once you've identified the root cause, you can start working on a plan to address it.
2. **Prioritize your tasks**: Make a to-do list and prioritize your tasks based on importance and urgency. Focus on completing the most critical tasks first, and then